In [169]:
from read_datasetBreakfast import load_data, read_mapping_dict, load_data_cust
import os
import torch
import seaborn as sn

COMP_PATH = 'C:\\Users\\cherl\\Desktop\\Masters\\CS5242 Neural Networks and Deep Learning\\Project'
''' 
training to load train set
test to load test set
'''
split = 'training'
#split = 'test'
train_split =  os.path.join(COMP_PATH, 'splits/train.split1.bundle') #Train Split
test_split  =  os.path.join(COMP_PATH, 'splits/test.split1.bundle') #Test Split
GT_folder   =  os.path.join(COMP_PATH, 'groundTruth/') #Ground Truth Labels for each training video 
DATA_folder =  os.path.join(COMP_PATH, 'data/') #Frame I3D features for all videos
mapping_loc =  os.path.join(COMP_PATH, 'splits/mapping_bf.txt') 

actions_dict = read_mapping_dict(mapping_loc)
if  split == 'training':
    data_feat_train, data_labels = load_data(train_split, actions_dict, GT_folder, DATA_folder, datatype = split) #Get features and labels
    data_feat_train_cust, data_labels_cust = load_data_cust(train_split, actions_dict, GT_folder, DATA_folder, datatype = split) #Get features and labels
if  split == 'test':
    data_feat_test = load_data(test_split, actions_dict, GT_folder, DATA_folder, datatype = split) #Get features only

#'''
#Write Code Below
#Pointers
#Need to load the segments.txt file for segments for test videos 
#Output the CSV in correct format as shown in Evaluation Section
#Id corresponds to the segments in order. 
#Example - 30-150 = Id 0
#          150-428 = Id 1
#          428-575 = Id 2
#Category is the Class of the Predicted Action
#'''

Finish Load the Training data and labels!!!
Finish Load the Training data and labels!!!


In [352]:
#Print info on the data
print("Number of Training Data: {}".format(len(data_feat_train)))
print("Number of Training Data Labels: {}".format(len(data_labels)))
print("Size of Sample Training Data #0: {}".format(data_feat_train[0].size()))

#Testing Data
try:
    print("Number of Test Data: {}".format(len(data_feat_test)))
    print("% of Training Data: {}".format(len(data_feat_test)/(len(data_feat_train)+len(data_feat_test))*100))
except NameError:
    print("Testing Data not loaded")

Number of Training Data: 199
Number of Training Data Labels: 199
Size of Sample Training Data #0: torch.Size([1595, 400])
Testing Data not loaded


In [354]:
#Preparing the data by splitting video into segments
from torch.utils.data import TensorDataset, DataLoader

segments = []
segment_labels = []
for i, video in enumerate(data_feat_train):
    frame_num = 0
    while frame_num < len(video):
        #Skip SIL frames
        if data_labels_cust[i][frame_num] > 0:
            start_index = frame_num # start index of segment
            end_index = frame_num # end index of segment
            start_label = data_labels_cust[i][frame_num] # label of segment
            for index in range(start_index + 1, len(video)):
                if data_labels_cust[i][index] == start_label:
                    frame_num=index + 1
                    end_index=index
                else:
                    frame_num=index + 1
                    break
            segment = video[start_index:end_index]
            segments.append(segment)
            segment_label = []
            segment_label.append(start_label)
            segment_labels.append(segment_label)
            #print(segment.size())
            #print("start {}, end {}".format(start_index, end_index))
        else:
            frame_num += 1
#print(segment_labels)

# Making all the segments equal length
standardized_segments = []
target_segment_length = 300 # Standard length to convert all segments to. Tune as needed.

for segment in segments:
    if len(segment) > target_segment_length:
        # Truncate segment
        standardized_segment = segment[0:target_segment_length]
        standardized_segments.append(standardized_segment.float())
    elif len(segment) == target_segment_length:
        standardized_segment = segment
        standardized_segments.append(standardized_segment.float())
    else:
        # Pad segment
        standardized_segment = torch.zeros(target_segment_length, 400)
        standardized_segment[0:len(segment)] = segment
        standardized_segments.append(standardized_segment.float())
    #print(standardized_segment.size())

print("Number of segments : {}".format(len(standardized_segments)))

Number of segments : 1042


In [362]:
# Creating data loaders from data
batch_size = 10
train_valid_split = 500
valid_end = 600
standardized_segments_tensor = torch.stack(standardized_segments)
segment_labels_tensor = torch.tensor(segment_labels)

train_data = TensorDataset(standardized_segments_tensor[:train_valid_split], segment_labels_tensor[:train_valid_split])
valid_data = TensorDataset(standardized_segments_tensor[train_valid_split:valid_end], segment_labels_tensor[train_valid_split:valid_end])

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=batch_size, shuffle=True)
    
print("DataLoaders created")

DataLoaders created


In [392]:
import torch.nn as nn
import torch.nn.functional as F

#Define the neural network modules

class mynet(nn.Module):
    def __init__(self):
        super(mynet, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 10, stride=5).double()
        self.pool = nn.MaxPool2d(5, stride=5).double()
        self.fc1 = nn.Linear(target_segment_length * 16, 47).double()
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        return x

class myGruNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers):
        super(myGruNet, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        self.gru = nn.GRU(input_dim, hidden_dim, n_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        
    def forward(self, x, h):
        out, h = self.gru(x, h)
        out = self.fc(self.relu(out[:,-1]))
        return out, h
    
    #Initialize the hidden layer
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device)
        return hidden

class myRnn(nn.Module):
    def __init__(self, input_size, hidden_dim, output_size, n_layers=1):
        super(myRnn, self).__init__()

        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        self.rnn = nn.RNN(input_size, hidden_dim, n_layers, batch_first=True)   
        self.fc = nn.Linear(hidden_dim, output_size)

    def forward(self, input):
         
        batch_size = input.size(0)

        # Initializing hidden state for first input using method defined below
        hidden = self.init_hidden(batch_size)
        
        # Passing in the input and hidden state into the model and obtaining outputs
        out, hidden = self.rnn(input, hidden)
        
        # Reshaping the outputs such that it can be fit into the fully connected layer
        out = out.contiguous().view(-1, self.hidden_dim)
        out = self.fc(out)
        
        return out, hidden

    def init_hidden(self, batch_size):
        return torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(device)
    
class myLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers, dropout_rate):
        super().__init__()
        self.n_layers = n_layers   # number of LSTM layers 
        self.hidden_dim = hidden_dim   # number of hidden nodes in LSTM
        self.dropout = dropout_rate
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, n_layers, batch_first = True, dropout=dropout_rate)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.sigmoid = nn.Sigmoid()
        
    def forward (self, input):
        #print("Input shape : {}".format(input.shape))
        #print(input[0])
        #print(input[1])
        lstm_out, h = self.lstm(input)
        #print("LSTM shape : {}".format(lstm_out.shape))
        #lstm_out = lstm_out.contiguous()#.view(-1, self.hidden_dim)
        #print("LSTM shape contiguous: {}".format(lstm_out.shape))
        
        fc_out = self.fc(lstm_out[:,-1:,:])
        #print("FC shape : {}".format(fc_out.shape))
        
        sigmoid_out = self.sigmoid(fc_out)
        #sigmoid_out = sigmoid_out.view(batch_size, input.shape[1], -1)
        #print("Sigmoid out shape : {}".format(sigmoid_out.shape))
        
        #sigmoid_last = sigmoid_out[:, -1,:] # Final output of RNN
        #print("Sigmoid last shape : {}".format(sigmoid_last.shape))
        return sigmoid_out, h
    
    def init_hidden (self, batch_size, device):         
        weights = next(self.parameters()).data
        h = (weights.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device),
             weights.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device))        
        return h

In [358]:
# Helper functions to run the training
import numpy as np

def cal_acc(probs, target):
    # probs: probability that each image is labeled as 1
    # target: ground truth label
    with torch.no_grad():
        prediction = torch.argmax(probs, axis=-1)    
        acc = torch.sum(prediction == target)
        return acc.item() / len(target) * 100

def train_one_pass_rnn(network, optim, device, criterion):
  # Input: Network and Optimizer
  # Output: Averge accuracy , Avergae loss in the pass
    network.train()
    acc_one_pass = []
    loss_one_pass = []
    for i, segment in enumerate(standardized_segments):
        optim.zero_grad()
        X = torch.tensor(segment).unsqueeze(dim=0).float().to(device)
        y = torch.ones(X.size()[1])
        y = torch.Tensor.new_full(y, y.size(), segment_labels[i][0], dtype=int, device=device, requires_grad=True)
        out, h = network(X)
        loss = criterion(out, y)
        loss.backward()
        optim.step()
    print(loss)

def train_one_pass_lstm(network, optim, device, criterion):
    # Input: Network and Optimizer
    # Output: Averge accuracy , Avergae loss in the pass
    network.train()
    
    h = network.init_hidden(batch_size, device)
    loss_one_pass = []
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        labels = labels.transpose(1,0).squeeze()
        h = tuple([each.data for each in h])   
        optim.zero_grad()
        out, h = network(inputs)
        loss = criterion(out.squeeze(), labels)
        loss_one_pass.append(loss.item())
        loss.backward()
        optim.step()
    #print(np.mean(loss_one_pass))
    return np.mean(loss_one_pass)

In [405]:
#Run the training

if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')

epochs = 90

# Model paramters
input_dim = 400 # number of features in each frame
hidden_dim = 150
output_dim = 47
n_layers = 1
dropout_rate = 0.25
    
#net = myGruNet(input_dim, hidden_dim, output_dim, n_layers).to(device)
#net = myRnn(input_dim, hidden_dim, output_dim).to(device)
net = myLSTM(input_dim, hidden_dim, output_dim, n_layers, dropout_rate).to(device)
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()#nn.BCELoss()
    
for epoch in range(epochs):
    train_loss = train_one_pass_lstm(net, optimizer, device, criterion)
    if epoch % 10 == 0:
        print(train_loss)
        
print("Finished training")

3.7605560541152956
3.43256010055542
3.309905686378479
3.2684721183776855
3.227519769668579
3.2030724763870237
3.1711995458602904
3.1635423612594606
3.1448573160171507
Finished training


In [407]:
# validation
net.eval()
valid_losses = []
valid_accuracy = []

v_h = net.init_hidden(batch_size, device)

for v_inputs, v_labels in valid_loader:
    v_inputs, v_labels = v_inputs.to(device), v_labels.to(device)
    v_h = tuple([each.data for each in v_h])
    
    v_labels = v_labels.transpose(1,0).squeeze()
    v_output, v_h = net(v_inputs)
    
    v_loss = criterion(v_output.squeeze(), v_labels)
    valid_losses.append(v_loss.item())

    with torch.no_grad():
        prediction = torch.argmax(v_output, axis=2)    
        #print(prediction.flatten())
        #print(v_labels)
        acc = torch.sum(prediction.flatten() == v_labels)
        batch_accuracy = acc.item() / len(v_labels) * 100
        valid_accuracy.append(batch_accuracy)     
        #print(batch_accuracy)
        
print("Validation Loss: {:.4f}".format(np.mean(valid_losses)), "Validation Accuracy: {:.4f}".format(np.mean(valid_accuracy)))


Validation Loss: 3.3319 Validation Accuracy: 19.0000
